In [1]:
import psycopg2 as psy
import sqlalchemy
import numpy as np
import pandas as pd
import time
import random
import datetime

In [2]:
#test the connection with default mimic build options
try:
    connection = psy.connect(user = "postgres",
                                  password = "postgres",
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "mimic")
    cursor = connection.cursor()
    # Print PostgreSQL Connection properties
    print ( connection.get_dsn_parameters(),"\n")
    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")
except (Exception, psy.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

{'user': 'postgres', 'dbname': 'mimic', 'host': '127.0.0.1', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.2 on x86_64-apple-darwin18.5.0, compiled by Apple LLVM version 10.0.1 (clang-1001.0.46.4), 64-bit',) 

PostgreSQL connection is closed


In [3]:
try:
    connection = psy.connect(user = "postgres",
                                  password = "postgres",
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "mimic")
    cursor1 = connection.cursor()
    postgreSQL_select_Query = "select distinct mimiciii.patients.dob, mimiciii.patients.subject_id, mimiciii.admissions.admittime from mimiciii.patients, mimiciii.pneumonia_bal_hadm, mimiciii.admissions where mimiciii.patients.subject_id = mimiciii.pneumonia_bal_hadm.subject_id and mimiciii.admissions.hadm_id = mimiciii.pneumonia_bal_hadm.hadm_id"
    cursor1.execute(postgreSQL_select_Query)
    patient_dob = cursor1.fetchall()
    age = []
    for item in patient_dob:
        item = list(item)
        pt_age =(((item[2]-item[0]).days)/365)
        if pt_age > 200 :
            pt_age = 95
            item.append(pt_age)
            item= tuple(item)
            age.append(item)
        else:
            item.append(pt_age)
            item=tuple(item)
            age.append(item)
    def set_age(list):
        cursor2 = connection.cursor()
        for row in list:
            sql_update_query = "Update mimiciii.redcap_distinct set pt_age = %s where mimiciii.redcap_distinct.subject_id = %s"
            age = float(row[3])
            ident = int(row[1])
            cursor2.execute(sql_update_query, (age, ident))
        connection.commit()
    set_age(age)
    cursor3 = connection.cursor()
    sql_select_bals = "Select bal.chartdate, bal.subject_id, dense_rank() over (partition by bal.subject_id order by bal.chartdate asc) as bal_order from mimiciii.bal, mimiciii.pneumonia_bal_hadm_visit where bal.subject_id = pneumonia_bal_hadm_visit.subject_id and pneumonia_bal_hadm_visit.visit = 1 order by bal.subject_id, bal_order"
    cursor3.execute(sql_select_bals)
    patient_bal_dt = cursor3.fetchall()
    cursor4=connection.cursor()
    for row in patient_bal_dt:
        sql_update_bal_dt = "Update mimiciii.redcap_distinct set initial_bal_dt = %s, study_enroll_dt = %s where redcap_distinct.subject_id = %s and %s = 1"
        bal_dt = row[0]
        sub_id = row[1]
        order = row[2]
        cursor4.execute(sql_update_bal_dt, (bal_dt, bal_dt, sub_id, order))
    connection.commit()
    cursor5=connection.cursor()
    sql_select_admit = "Select admissions.admission_location, admissions.subject_id from mimiciii.admissions, mimiciii.pneumonia_bal_hadm_visit where admissions.hadm_id = pneumonia_bal_hadm_visit.hadm_id and pneumonia_bal_hadm_visit.visit =1"
    cursor5.execute(sql_select_admit)
    admits=cursor5.fetchall()
    cursor6=connection.cursor()
    for row in admits:
            row = list(row)
            if row[0] == 'PHYS REFERRAL/NORMAL DELI':
                row.append(5)
            elif row[0] == 'TRANSFER FROM HOSP/EXTRAM':
                row.append(3)
            elif row[0] == 'CLINIC REFERRAL/PREMATURE':
                row.append(5)
            elif row[0] == 'TRANSFER FROM SKILLED NUR':
                row.append(4)
            elif row[0] == 'TRANSFER FROM OTHER HEALT':
                row.append(5)
            else:
                row.append(0)
            if row[2] == 3:
                radio = 1
            elif row[2] == 4:
                radio = 1
            elif row[2] == 5:
                radio = 1
            else:
                radio = 0
            row = tuple(row)
            hctype = int(row[2])
            idnum = int(row[1])
            sql_update_admits = "Update mimiciii.redcap_distinct set pt_admit_hcfacility = %s, pt_hcfacility_type = %s where redcap_distinct.subject_id = %s"
            cursor6.execute(sql_update_admits, (radio, hctype, idnum))
    connection.commit()
    cursor7 = connection.cursor()
    sql_update_mech_vent = "Update mimiciii.redcap_distinct set pt_mech_vent = 1"
    cursor7.execute(sql_update_mech_vent)
    connection.commit()
finally:
    # closing database connection
    if (connection):
            cursor1.close()
            connection.close()
    print("PostgreSQL connection is closed \n")
                
    
#except (Exception, psy.Error) as error:
            #print(error)


PostgreSQL connection is closed 



In [4]:
try:
    connection = psy.connect(user = "postgres",
                                  password = "postgres",
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "mimic")
    cursor8 = connection.cursor()
    update_query = "Update mimiciii.redcap_distinct set pt_hosp_90d = 0, pt_lastdc = 0"
    cursor8.execute(update_query)
    connection.commit()
    cursor9 = connection.cursor()
    select_query = "Select mimiciii.readmits_redcap_weeks.subject_id, weeks from mimiciii.readmits_redcap_weeks where visits = 2"
    cursor9.execute(select_query)
    readmits = cursor9.fetchall()
    cursor10 = connection.cursor()
    for row in readmits:
        row = list(row)
        sql_update_readmits = "Update mimiciii.redcap_distinct set pt_hosp_90d = 1, pt_lastdc = %s where redcap_distinct.subject_id = %s"
        idnum = row[0]
        weeks = int(row[1])
        cursor10.execute(sql_update_readmits, [weeks, idnum])
    connection.commit()
    cursor11 = connection.cursor()
    update_query = "Update mimiciii.redcap_distinct set pt_carrest=0"
    cursor11.execute(update_query)
    connection.commit()
    cursor12=connection.cursor()
    select_carrest = "Select mimiciii.pneumonia_bal_primary_visit.subject_id from mimiciii.pneumonia_bal_primary_visit, mimiciii.diagnoses_icd where pneumonia_bal_primary_visit.hadm_id = diagnoses_icd.hadm_id and diagnoses_icd.icd9_code = '4275'"
    cursor12.execute(select_carrest)
    carrest = cursor12.fetchall()
    cursor13 = connection.cursor()
    for row in carrest:
        update_carrest = "Update mimiciii.redcap_distinct set pt_carrest = 1 where redcap_distinct.subject_id = %s"
        row = list(row)
        idsub = row[0]
        cursor13.execute(update_carrest, [idsub,])
    connection.commit()
    
    cursor14=connection.cursor()
    update_query = 'Update mimiciii.redcap_distinct set pt_immunocomp = 0'
    cursor14.execute(update_query)
    update_query_immunotype = 'Update mimiciii.redcap_distinct set type_immunocomp =0'
    cursor14.execute(update_query_immunotype)
    connection.commit()
    cursor15 = connection.cursor()
    select_immuno = "Select mimiciii.pneumonia_bal_primary_visit.subject_id, mimiciii.diagnoses_icd.icd9_code from mimiciii.pneumonia_bal_primary_visit, mimiciii.diagnoses_icd where pneumonia_bal_primary_visit.hadm_id = diagnoses_icd.hadm_id and (diagnoses_icd.icd9_code = '20400' or diagnoses_icd.icd9_code = '042' or diagnoses_icd.icd9_code = '27901' or diagnoses_icd.icd9_code = '27900' or diagnoses_icd.icd9_code = '27902' or diagnoses_icd.icd9_code = '27903' or diagnoses_icd.icd9_code = '27904' or diagnoses_icd.icd9_code = '27905' or diagnoses_icd.icd9_code = '20280' or diagnoses_icd.icd9_code = '20020' or diagnoses_icd.icd9_code = '20000' or diagnoses_icd.icd9_code = '20190' or diagnoses_icd.icd9_code = '20070' or diagnoses_icd.icd9_code = '20060' or diagnoses_icd.icd9_code = '20010' or diagnoses_icd.icd9_code = '20040' or diagnoses_icd.icd9_code = '20080' or diagnoses_icd.icd9_code = '20210' or diagnoses_icd.icd9_code = '20270' or diagnoses_icd.icd9_code = '20300' or diagnoses_icd.icd9_code = 'v420' or diagnoses_icd.icd9_code = 'v421' or diagnoses_icd.icd9_code = 'v422' or diagnoses_icd.icd9_code = 'v423' or diagnoses_icd.icd9_code = 'v424' or diagnoses_icd.icd9_code = 'v425' or diagnoses_icd.icd9_code = 'v426' or diagnoses_icd.icd9_code = 'v427' or diagnoses_icd.icd9_code = 'v428' or diagnoses_icd.icd9_code = 'v429' or diagnoses_icd.icd9_code = 'v4282' or diagnoses_icd.icd9_code = 'v4283' or diagnoses_icd.icd9_code = 'v4284' or diagnoses_icd.icd9_code = 'v4289') "
    cursor15.execute(select_immuno)
    immuno = cursor15.fetchall()
    cursor16 = connection.cursor()
    for row in immuno:
        update_immunocomp = 'Update mimiciii.redcap_distinct set pt_immunocomp = 1 where redcap_distinct.subject_id = %s'
        update_immuno = "Update mimiciii.redcap_distinct set type_immunocomp = %s where redcap_distinct.subject_id = %s"
    
        row = list(row)
        
        if row[1]=='20400':
            row.append(1)
        if row[1]=='042':
            row.append(7)
        if row[1]=='27901' or '27900' or '27902' or '27903' or '27904' or '27905':
            row.append(8)
        if row[1]=='20280' or '20020' or '20000' or '20190' or '20070' or '20060' or '20010' or '20040' or '20080' or '20210':
            row.append(9)
        if row[1]=='v4289' or 'v4284' or 'v4283' or 'v429' or 'v428' or 'v427' or 'v426' or 'v425' or 'v424' or 'v423' or 'v422' or 'v421' or 'v420':
            row.append(14)
        if row[1]=='v4282':
            row.append(15)
        
        idsub = row[0]
        immunocomp = row[2]
        cursor16.execute(update_immunocomp, [idsub,])
        cursor16.execute(update_immuno, [immunocomp, idsub, ])
    connection.commit()
    
    cursor99=connection.cursor()
    select_meds='Select mimiciii.pneumonia_bal_primary_visit.subject_id, '
    
    cursor17 = connection.cursor()
    select_query_race= 'Select mimiciii.pneumonia_bal_primary_visit.subject_id, mimiciii.admissions.ethnicity from mimiciii.pneumonia_bal_primary_visit, mimiciii.admissions where pneumonia_bal_primary_visit.hadm_id=admissions.hadm_id'
    cursor17.execute(select_query_race)
    race = cursor17.fetchall()
    
    for row in race:
        row = list(row)
        update_race = 'Update mimiciii.redcap_distinct set race_desc = %s where redcap_distinct.subject_id =%s'
        idsub=row[0]
        race=row[1]
        cursor17.execute(update_race, [race, idsub,])
    connection.commit()
    
    cursor18=connection.cursor()
    select_query_alive= "Select mimiciii.pneumonia_bal_primary_visit.subject_id, mimiciii.admissions.hospital_expire_flag from mimiciii.pneumonia_bal_primary_visit, mimiciii.admissions where pneumonia_bal_primary_visit.hadm_id=admissions.hadm_id"
    cursor18.execute(select_query_alive)
    alive = cursor18.fetchall()
    
    for row in alive:
        row = list(row)
        if row[1]==1:
            row.append(0)
        if row[1]==0:
            row.append(1)
        update_alive= "Update mimiciii.redcap_distinct set dc_alive = %s where redcap_distinct.subject_id=%s"
        idsub=row[0]
        alive=row[2]
        cursor18.execute(update_alive, [alive, idsub,])
    connection.commit()
    
    cursor19=connection.cursor()
    select_discharge_date="Select mimiciii.pneumonia_bal_primary_visit.subject_id, mimiciii.admissions.hospital_expire_flag, mimiciii.admissions.dischtime, mimiciii.admissions.admittime from mimiciii.pneumonia_bal_primary_visit, mimiciii.admissions where mimiciii.pneumonia_bal_primary_visit.hadm_id=mimiciii.admissions.hadm_id"
    cursor19.execute(select_discharge_date)
    discharge=cursor19.fetchall()
    
    for row in discharge:
        row=list(row)
        if row[1]==0:
            update_discharge = "Update mimiciii.redcap_distinct set dc_dt = %s, dc_study_day = %s where redcap_distinct.subject_id=%s"
            discharge=row[2]
            admit=row[3]
            idsub=row[0]
            study_day=int((discharge-admit).days)
            cursor19.execute(update_discharge, [discharge, study_day, idsub,])
        if row[1]==1:
            update_discharge = "Update mimiciii.redcap_distinct set death_dt = %s, death_study_day = %s where redcap_distinct.subject_id=%s"
            discharge=row[2]
            admit=row[3]
            idsub=row[0]
            study_day=int((discharge-admit).days)
            cursor19.execute(update_discharge, [discharge, study_day, idsub,])
    connection.commit()
finally:
    # closing database connection
    if (connection):
            cursor1.close()
            connection.close()
    print("PostgreSQL connection is closed \n")    

PostgreSQL connection is closed 

